In [1]:
"""
Embed tokens in the dataset
"""
# path to raw dataset
raw_dataset_path = './stlawu-webpages.jsonl'
print(raw_dataset_path)

./stlawu-webpages.jsonl


In [5]:
from datasets import load_dataset

raw_dataset = load_dataset('json', data_files=raw_dataset_path, split='train')
raw_dataset



Dataset({
    features: ['url', 'html_doc'],
    num_rows: 8160
})

In [6]:
from transformers import AutoTokenizer, AutoModel

model_ckpt = "sentence-transformers/multi-qa-mpnet-base-dot-v1"
tokenizer = AutoTokenizer.from_pretrained(model_ckpt)
model = AutoModel.from_pretrained(model_ckpt)

In [11]:
# import torch

# device = torch.device("cuda")
# model.to(device)

In [10]:
tokenized_text = tokenizer(raw_dataset[0]['html_doc'])
tokenized_text.input_ids[:10]

[0, 1030, 1003, 9990, 13878, 16133, 1032, 1030, 16133, 16105]

In [13]:
def get_embeddings(text_list):
    encoded_input = tokenizer(
        text_list, padding=True, truncation=True, return_tensors="pt"
    )
    # encoded_input = {k: v.to(device) for k, v in encoded_input.items()}
    model_output = model(**encoded_input)
    # return cls_pooling(model_output)
    return model_output


In [16]:
get_embeddings(raw_dataset['html_doc'][0])

BaseModelOutputWithPooling(last_hidden_state=tensor([[[-7.8214e-02, -3.3229e-01, -3.1041e-01,  ...,  1.5539e-01,
          -3.4083e-01, -1.4064e-01],
         [-1.4663e-02, -5.2548e-01, -3.4385e-01,  ...,  4.8605e-02,
          -4.9307e-01, -1.5615e-01],
         [-3.2722e-02, -5.5539e-01, -3.6671e-01,  ...,  6.4318e-02,
          -4.6556e-01, -1.4308e-01],
         ...,
         [-2.5340e-02,  2.6188e-01, -2.8428e-01,  ...,  2.6971e-01,
          -2.7826e-01, -1.8083e-04],
         [-7.3161e-02, -1.5915e-01, -2.3299e-01,  ...,  2.2992e-01,
          -2.0738e-01, -1.5243e-01],
         [-6.5796e-02, -3.9216e-01, -2.9852e-01,  ...,  1.4378e-01,
          -3.3341e-01, -1.6220e-01]]], grad_fn=<NativeLayerNormBackward0>), pooler_output=tensor([[-1.0435e-01, -1.0654e-01,  9.1496e-02, -1.7933e-01, -8.9404e-02,
          1.1848e-01, -4.4903e-02,  6.4997e-02, -2.4272e-01, -1.1525e-02,
          1.2531e-02, -2.9322e-01,  1.7162e-01, -6.7447e-02,  6.1314e-02,
         -2.2843e-01, -2.5585e-01,  